# Part 3: Running Cortex Certifai Scan on models deployed as service on Certifai Pro

# Typical flow for a Data Scientist and ML Engineer during Development when they want to delegate the Scan activity with larger dataset and scan.

- Detailed analysis
- Setup Baseline

## Documentation

Visit the Certifai Documentation here https://cognitivescale.github.io/cortex-certifai/docs/about

## Prerequisites

Note: Please complete **part2_certifai_workshop_scan_registered_models.ipynb** and obtain **Service keys and Endpoints** to continue.


In [1]:
from pathlib import Path
pwd = !pwd
my_path = str(Path(pwd[0]).parents[0])
certifai_toolkit_path = my_path  + '/certifai_toolkit_1.3.3'
certifai_toolkit_path

'/Users/skottaram/Documents/runtimes/certifai_toolkit_1.3.3'

## 1. Test the two Models deployed in ACI (AZURE CONTAINER INSTANCE) 
The following cells load sample data and test the Service Keys and Endpoints obtained from `part2_certifai_workshop_scan_registered_models.ipynb`

In [2]:
service_logistic_uri  = 'replace'
service_logistic_key  = 'replace'

service_svm_uri       = 'replace'
service_svm_key       = 'replace'

In [3]:
# create json test data sample(from csv)

import json
sample_input = json.dumps({
"payload": {
    "instances": [
        [
            "... < 0 DM",
            6,
            "critical account/ other credits existing (not at this bank)",
            "radio/television",
            1169,
            "unknown/ no savings account",
            ".. >= 7 years",
            4,
            "male : single",
            "others - none",
            4,
            "real estate",
            "> 25 years",
            "none",
            "own",
            2,
            "skilled employee / official",
            1,
            "phone - yes, registered under the customers name",
            "foreign - yes"
        ]
    ]
}
})
sample_input

'{"payload": {"instances": [["... < 0 DM", 6, "critical account/ other credits existing (not at this bank)", "radio/television", 1169, "unknown/ no savings account", ".. >= 7 years", 4, "male : single", "others - none", 4, "real estate", "> 25 years", "none", "own", 2, "skilled employee / official", 1, "phone - yes, registered under the customers name", "foreign - yes"]]}}'

In [4]:
import requests
import json

headers_svm = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {service_svm_key}'          
          }
headers_logistic = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {service_logistic_key}'          
          }

response = requests.post(
    service_svm_uri, data=sample_input, headers=headers_svm)
print('SVM Model Endpoint Inference Test')
print(response.status_code)
print(response.elapsed)
print(response.json())

print('Logistic Regression Model Endpoint Inference Test')
response = requests.post(
    service_logistic_uri, data=sample_input, headers=headers_logistic)
print(response.status_code)
print(response.elapsed)
print(response.json())

SVM Model Endpoint Inference Test
200
0:00:00.150167
{'payload': {'predictions': [1]}}
Logistic Regression Model Endpoint Inference Test
200
0:00:00.146625
{'payload': {'predictions': [1]}}


## 2. Use Cortex Certifai Client CLI to Configure Remote Certifai Pro VM
Use the `certifai` CLI tool to configure remote access to the Certifai Pro VM

**Note:** Download certifai-kubeconfig.json from your instance of Certifai Pro.
Log into your console and look for "Download kubeconfig" option in the toolbar.

detailed instructions available here:
https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup

#### CLI commands

```
# remove flower brackets if not in a jupyter notebook cell
certifai remote config --file certifai-kubeconfig.json --alias {remote_alias} 
```

In [5]:
remote_alias = 'cpro-az'
!certifai remote config --file certifai-kubeconfig.json --alias {remote_alias}


Checking for access to Kubernetes cluster with context - certifai-pro
Connection to cluster succeeded, found API - v1
Updating alias - cpro-az
Configuration updated from - certifai-kubeconfig.json


## 3. Construct Certifai Scan Definitions for Binary Classification models

Use the `certifai-scanner` python package to build a scan definition for the SVM and Logistic Regression models (via the Service Endpoints created earlier).

Configure scan definition parameters include (mandatory):

1. Prediction Task Outcomes and Values
2. Model Details (names, endpoints and more)
3. Datasets to evaluate the models on

Configure optional scan definition parameters that depend on the desired evaluation reports. Evaluation types include:

1. Fairness
2. Robustness
3. Explainability

In [6]:
# make sure certifai package was installed correctly
!certifai --version

Certifai version: 1.3.3
Scanner build: 1.3.3-159-ga98e943b


In [7]:
# necessary imports for creating a scan

from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiModel, CertifaiModelMetric,
                                      CertifaiDataset, CertifaiGroupingFeature, CertifaiDatasetSource,
                                      CertifaiPredictionTask, CertifaiTaskOutcomes, CertifaiOutcomeValue)
from certifai.scanner.report_utils import scores, construct_scores_dataframe


### Define Cortex Certifai Task Type

- `CertifaiTaskOutcomes` : Cortex Certifai supports classification as well as regression models. The task type of this tutorial is binary-classification (e.g. predicting whether a loan should be granted or not)
- `CertifaiOutcomeValue` : Define the different outcomes possible from the model predictions. These models predict either 1(loan granted) or 2(loan denied)

**NOTE**: Please refer to [Certifai Api Docs](https://cognitivescale.github.io/cortex-certifai/certifai-api-ref/certifai.scanner.builder.html) for more details

In [28]:
# Create the scan object from scratch using the ScanBuilder class with tasks and outcomes

# First define the possible prediction outcomes
task = CertifaiPredictionTask(CertifaiTaskOutcomes.classification(
    [
        CertifaiOutcomeValue(1, name='Loan granted', favorable=True),
        CertifaiOutcomeValue(2, name='Loan denied')
    ]),
    prediction_description='Determine whether a loan should be granted')

#  create a certifai scan object and add the certifai task created above
scan = CertifaiScanBuilder.create('certifai_pro_demo',
                                  prediction_task=task)

scan

### Add Logistic and SVM Models to the Scan definition

Additional parameters that maybe provided to the `CertifaiModel` class can be gleaned from the [API Reference for CertifaiModel](https://cognitivescale.github.io/cortex-certifai/certifai-api-ref-1.2.14/certifai.scanner.builder.html#certifai.scanner.builder.CertifaiModel)

or `?CertifaiModel`

In [29]:
# Create a Certifai Model Object using the web service (from earlier) by passing the deployed web service url
first_model = CertifaiModel('SVM',
                            predict_endpoint=service_svm_uri)
scan.add_model(first_model)

second_model = CertifaiModel('logistic',
                            predict_endpoint=service_logistic_uri)
scan.add_model(second_model)

# Add corresponding model headers for service authentication and content-type

# add the default headers applicable to all models
scan.add_model_header(header_name='Content-Type',header_value='application/json')

# add defined headers corresponding to auth keys for respective model services
scan.add_model_header(header_name='Authorization', header_value=f'Bearer {service_svm_key}', model_id='SVM')
scan.add_model_header(header_name='Authorization', header_value=f'Bearer {service_logistic_key}', model_id='logistic')



## 4. Uploading Required Datasets into the Azure Storage Account Container

Use the `azure-storage-blob` python package to upload the evaluation dataset for the Certifai scan to the Azure storage account blob container configured in Step 1 of the Certifai Pro VM setup process

### Upload the Dataset to Azure Blob Storage

Get the connection string for the storage account that holds the container called {az_container_name} in the cell below. You can obtain the connection string by following the [Azure guide here](https://docs.microsoft.com/en-us/azure/storage/common/storage-configure-connection-string).

Ensure that the storage account and container used here match the values you used at the beginning of this notebook to setup your Certifai Pro instance.

You can get storage blob credentials by visiting Access keys -> Connection String from your Storage Account's page on the Azure Portal.

In [ ]:
# upload our eval dataset to the blob storage container
az_container_name = 'replace' # this container should already exist. You can create one from the Azure Portal

import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# set credentials for azure storage account
az_credentials = 'connectionString'


client = BlobServiceClient.from_connection_string(az_credentials)
german_credit_eval_data_file = "data/german_credit_eval.csv"
az_german_credit_blob_name = 'az-pro-example/german_credit_eval.csv'

# upload our evaluation dataset to an Azure Blob Storage Account Container.
blob_client = client.get_blob_client(container=az_container_name, blob=az_german_credit_blob_name)
with open(german_credit_eval_data_file, 'rb') as f:
    blob_client.upload_blob(f)

### Add the Evaluation Dataset to Scan Definition

Add the evaluation dataset to be used by Cortex Certifai to evaluate the model against the scan definition

In [31]:
# create an evaluation object and pass the evaluation dataset(csv) here 
eval_dataset = CertifaiDataset('evaluation',
                               CertifaiDatasetSource.csv(url=f'abfs://{az_container_name}/{az_german_credit_blob_name}'))
scan.add_dataset(eval_dataset)

### Configure Model Fairness Evaluation

- Add `fairness` as an evaluation type to the scan object
- Create an `evaluation_dataset_id` to refer to the added evaluation datset

In [32]:
# Setup an evaluation for fairness on the above dataset using the model
# We'll look at disparity between groups defined by marital status and age
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('age'))
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('status'))
scan.add_evaluation_type('fairness')
scan.evaluation_dataset_id = 'evaluation'

In [33]:
# Because the dataset contains a ground truth outcome column which the model does not
# expect to receive as input we need to state that in the dataset schema (since it cannot
# be inferred from the CSV)
scan.dataset_schema.outcome_feature_name = 'outcome'

### Add Authorization Parameters to the Models
Use the following code block to update the scan definition that you constructed in the steps above with authorization headers needed to invoke your Azure hosted model endpoints

In [34]:
local_scan_definition_file = 'data/german_credit_scan_definition.yaml'
model_headers_template = f"""
model_headers:
  default:
  - name: Content-Type
    value: application/json
  - name: accept
    value: application/json
  defined:
  - model_id: SVM
    name: Authorization
    value: Bearer {service_svm_key}
  - model_id: logistic
    name: Authorization
    value: Bearer {service_logistic_key}
"""

with open(local_scan_definition_file, 'w') as f:
    scan.save(f)
# we also need to add the model headers section separately
with open(local_scan_definition_file, 'a') as f:
    f.write(model_headers_template)

## 5. Run a Remote Scan on Certifai Pro

In [35]:
reports_folder = f'abfs://{az_container_name}'
# run a remote scan
!certifai remote scan --alias cpro-az --definition-file data/german_credit_scan_definition.yaml --output {reports_folder}

Created certifai scan ID 11d4c78c9a1a


### Get Logs of the Remote Scan

In [43]:
!certifai remote logs -a cpro-az $(certifai remote list -a cpro-az | head -2 | tail -1 | cut -d' ' -f1)

2020-08-05 00:17:13,892 - root - INFO - Reading default config (fallback) from: /opt/app-root/lib/python3.6/site-packages/certifai/common/utils/default_certifai_config.ini
2020-08-05 00:17:13,892 - root - INFO - Read config marker: config['default']['marker'] = 0.1
2020-08-05 00:17:14,694 - root - INFO - Validating license...
2020-08-05 00:17:14,694 - root - INFO - License is valid - expires: n/a
2020-08-05 00:17:14,710 - root - INFO - Generated unique scan id: 7178b79af767
2020-08-05 00:17:14,710 - root - INFO - Validating input data...
2020-08-05 00:17:14,711 - root - INFO - Creating dataset with id: evaluation
2020-08-05 00:17:14,712 - azure.storage.common.storageclient - INFO - Client-Request-ID=03a80cba-d6b1-11ea-99a8-dea3e4cc26ae Outgoing request: Method=GET, Path=/certifai01302, Query={'restype': 'container', 'comp': 'list', 'prefix': 'az-pro-example', 'delimiter': '/', 'marker': None, 'maxresults': None, 'include': None, 'timeout': None}, Headers={'x-ms-version': '2019-02-02', 

### List the Scan Jobs on the Certifai Pro Remote Instance
Use the CLI command to list scan jobs for the configured `remote_alias`
```
certifai remote list -a <remote_alias>
```

In [42]:
# Check the status of the triggered remote scan job
!certifai remote list -a cpro-az

Scan ID        Total Reports   Completed Reports   Failed Reports   Active Reports   Progress   State      Scan Duration   
11d4c78c9a1a   4               4                   0                0                100        Complete   12m32s          
7c4651b38a27   4               4                   0                0                100        Complete   12m34s          
c11c2873fe77   4               4                   0                0                100        Complete   11m51s          


## View Reports from the Remote Scan

Once the remote scan's `COMPLETIONS` field says `1/1`, you can configure the Certifai Console to view the reports.

Go to the Certifai Console URL for the Certifai Pro VM instance you created earlier in this tutorial. 

1. Click the `User Icon` on the top right and select `Storage Settings` from the dropdown.
2. Update the `Scan Directory` field to the `reports_folder` variable configured in the previous cell. Omit `abfs://` while pasting this variable's value in the `Scan Directory` field. 
3. Save your settings and wait while the page reloads and loads reports from the remote scan. The scan will be available under the name `model_auth_demo`.


## Resource Cleanup


Delete the Certifai Pro VM instance created when you are done using it.

Azure ML pre-requisites Cleanup
 - Created and registered `logistic_model_azure` and `svm_model_azure` models to your Azure workspace -> Delete these. The exact names will be available from the output of the `azure_ml_service_keys.py` script. (From when you ran it earlier)
 - Created `german-credit-logistic-service` and `german-credit-svm-service` ACI (Azure Container Instance) webservices -> Delete these. The exact names will be available from the output of the `azure_ml_service_keys` script. (From when you ran it earlier)

- Once Cortex Certifai evaluation is complete, make sure to clear all Azure resources in order to avoid costs associated with running VMs and their associated resources on Azure
- Follow the [Azure Ml resource cleanup docs][https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-train#clean-up-resources]